<a href="https://colab.research.google.com/github/bforsbe/SK2534/blob/main/kinetics_seminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Kinetics in Molecular Systems — Seminar Notebook

**Today:**
- Connect thermodynamics (populations) and kinetics (rates).
- Compute rates from energy barriers using the Arrhenius equation, and explore temperature dependence as a way to estimate rates from experiments.
- Discover how rates combine for **serial** vs **parallel** barriers (students will *derive/discover* the rules).
- Combine these notions to understans the balance of folding and unfolding rates in a chevron plot.




## Recap: Boltzmann populations (Thermodynamics refresher)

At equilibrium, the probability (population) of being in microstate $i$ with energy $E_i$
is given by the Boltzmann distribution:
$$
p_i = \frac{e^{-E_i / (k_B T)}}{\sum_j e^{-E_j / (k_B T)}}
$$

- $k_B$ is the Boltzmann constant.
- Use these populations as the thermodynamic *equilibrium* — they **do not** contain time information about dynamic processes like folding or forward and reverse rates.

Let's plot the population of states, just to remind ourselves of the logic.


In [ ]:
#@title Compute regular Boltzmann populations
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Boltzmann constants and temperature
kB = 1.380649e-23  # J/K
R = 8.314462618  # J/mol/K
T = 300.0  # K

# Panel 1: Discrete states
energies_kjmol = np.array([0.0, 3.0, -2.0])  # kJ/mol
energies_discrete = energies_kjmol * 1000.0  # J/mol
probs_discrete = np.exp(-energies_discrete / (R * T))
probs_discrete /= probs_discrete.sum()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot discrete states
ax1.bar(range(len(energies_kjmol)), probs_discrete, color='skyblue')
ax1.set_xticks(range(len(energies_kjmol)))
ax1.set_xticklabels([f'State {i+1}\n(??? kJ/mol)' for i, e in enumerate(energies_kjmol)])
ax1.set_ylabel('Population')
ax1.set_title('Boltzmann Populations (Discrete States)')
ax1.grid(axis='y', linestyle=':')

# Panel 2: Continuous potential from Gaussian mixture
# Define parameters for two Gaussians (representing populations)
mu1, sigma1 = -2, 1  # Mean and standard deviation for the first Gaussian
mu2, sigma2 = 3, 1.5 # Mean and standard deviation for the second Gaussian
weight1, weight2 = 0.6, 0.4 # Weights for the mixture

x = np.linspace(-5, 8, 200)

# Calculate probability density from the mixture of two Gaussians
pdf = weight1 * norm.pdf(x, mu1, sigma1) + weight2 * norm.pdf(x, mu2, sigma2)

# Convert population (pdf) to potential energy using Boltzmann inversion
# P(E) = exp(-E/RT) => E = -RT * ln(P(E))
# We need to normalize the pdf to avoid ln(0) and to get meaningful relative energies.
# Let's set the minimum energy to 0 by shifting the potential.
min_pdf = np.min(pdf[pdf > 0]) # Find min non-zero pdf to avoid log(0)
pdf_normalized = pdf / min_pdf # Normalize relative to the minimum

# Calculate potential energy (in kJ/mol for easier comparison)
potential_kjmol = -R * T * np.log(pdf_normalized) / 1000.0

# Plot continuous potential
ax2.plot(x, potential_kjmol, label='Potential from Gaussian Mixture')
ax2.set_xlabel('Reaction Coordinate (arbitrary)')
ax2.set_ylabel('Potential Energy [kJ/mol]')
ax2.set_title('Continuous Potential and Population')
ax2.grid(True, linestyle=':')

# Create a secondary y-axis for the population plot
ax2b = ax2.twinx()
ax2b.plot(x, pdf, color='orange', linestyle='--', label='Population (Gaussian Mixture)')
ax2b.set_ylabel('Population Density')
ax2b.tick_params(axis='y', labelcolor='orange')

# Combine legends from both axes
lines, labels = ax2.get_legend_handles_labels()
lines2, labels2 = ax2b.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='best')


plt.tight_layout()
plt.show()

You can see that the lower the free energy, the higher the population. This **Boltzmann inversion** is evident especially in the continuous case.

**Question 1**:
> Sketch (in your mind or on paper), the three-state energy diagram of the left panel, and suggest approximate free energy differences between all states.


<details>
  <summary> Answer (click to expand)</summary>
  <p>
  State 1 is arbitrary, lets define it at E=0

  State 2 is lower in population, so higher in energy (E>0)

  State 3 is higher than both, so lower in in energy (E<0)
  </p>
</details>


## Single barrier

The three-state system defiend above is an example of a single barrier, much like we have seen before in the course. Now lets now try to acces the time-dependent characteristics of this system.

In a previous seminar we derived the metropolis criterion to accept a move to a higher state. We based that on a two-state system where the rates needed to balanace to fulfil the Bolztmann factors. We found that the probablity was
$$k = \exp\left(-\frac{\Delta H}{RT}\right)$$

where $\Delta H$ was the difference in energy, whereas the probability of going the other way (down in energy) was 100%. If we generalize this to a continuous landscape like the right panel in the plot above, it gets very complicated. We have to solve something called the **Fokker–Planck / Smoluchowski equation**, and take it in the **limit of low-temperature or high-barrier**, which leads to something called **Karmers classic rate formula**. We don't go into this here, but one neat thing is that through all these steps, the fundamental form of the expression is preserved, and the resulting expression encapsulates aspects about the landscape and other things in a simple frefactor, that we can call $A$. This is the then exactly the Arrhenius equation:

**Arrhenius (simple, empirical):**
$$k = A \cdot \exp\left(-\frac{E_a}{R T}\right)$$
- $A$ is the pre-exponential factor (frequency prefactor).
- $E_a$ is the activation energy (J/mol).

**Question 2**  
> There is nothing in the Arrhenius equation about time. How can we get time from it?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  The Arrhenius equation gives us the rate constant:
  
  $$
  k = A \, e^{-E_a / (RT)}
  $$
  
  The rate constant $k$ has units of **1/time** for a first-order process.  
  Thus, the characteristic timescale is simply:
  
  $$
  \tau = \frac{1}{k}
  $$
  
  where $\tau$ is the mean waiting time for the event (e.g., folding, reaction).
  </p>
  
  <p>
  The prefactor $A$ contains information about the attempt frequency (how often the system tries to cross the barrier).  
  This becomes more explicit in the <b>Eyring equation</b> from transition state theory:
  
  $$
  k = \frac{k_B T}{h} \, e^{-\Delta G^{\ddagger}/RT}
  $$
  
  Here, $\tfrac{k_B T}{h}$ has units of frequency (1/time), directly showing the link between the rate constant and a physical timescale.  
  </p>
  
  <p>
  In short: the Arrhenius form hides the timescale in the prefactor $A$, while the Eyring equation makes it explicit.
  </p>
</details>


## Arrhenius Plots

An **Arrhenius plot** is a way to show how rates depend on temperature, which will prove useful to estimate rates from experiment. There is a bit of math, but the purpose will become clear.

We start with the Arrhenius equation:
$$
k = A \, e^{-\frac{E_a}{RT}}
$$

Taking logs:
$$
\ln k = \ln A - \frac{E_a}{R}\frac{1}{T}
$$

So if we think of $1/T$ as a variable, this will just look like a line. We just need to plot $\ln(k)$ vs $1/T$:
- **Slope:** $-E_a/R$ (gives activation energy $E_a$)  
- **Intercept:** $\ln A$ (gives the prefactor $A$)  

### Experiments to measure k

So any experiment where we can measure a reaction rate at different temperatures will suffice.  
Here are a few common examples:

- **Stopped-flow**: two solutions are mixed very rapidly and the reaction progress is monitored in real time (ms–s). Great for fast folding/unfolding or enzyme reactions.  
- **UV–Vis spectroscopy (e.g. MTT assay)**: changes in absorbance report on product or reactant concentration over time. From the time trace you can fit and extract $k$.  
- **NMR kinetics**: peak intensities shift as reactants convert to products; following these changes gives the rate constant directly.

Repeat at several temperatures → plot $\ln k$ vs $1/T$ → obtain $E_a$ and $A$.



In [ ]:
#@title Estimating Ea from synthetic data using an Arrhenius plot
import numpy as np
import matplotlib.pyplot as plt

R = 8.314  # J/mol/K
Ea = 50e3  # activation energy, J/mol
A = 1e13   # pre-exponential factor, 1/s

# Temperature range
T = np.linspace(250, 500, 10)  # Kelvin
k = A * np.exp(-Ea/(R*T))      # rate constants

# Arrhenius plot: ln(k) vs 1/T
plt.figure(figsize=(6,4))
plt.plot(1/T, np.log(k), 'o-', label='data')
plt.xlabel('1/T (1/K)')
plt.ylabel('ln k')
plt.title('Arrhenius Plot')
plt.grid(True, ls=':')
plt.legend()
plt.show()

# Linear regression to extract Ea and A
coeffs = np.polyfit(1/T, np.log(k), 1)
slope, intercept = coeffs
Ea_fit = -slope * R
A_fit = np.exp(intercept)

print(f"Fitted Ea = {Ea_fit/1000:.2f} kJ/mol (true {Ea/1000:.2f})")
#print(f"Fitted A  = {A_fit:.2e} 1/s (true {A:.2e})")


## Serial barriers (A → I → B)

But a single transtion state seems crazy - protein folding must be a very complicated process, not just a two-state hop. Let's see what happens if we add just one more hop, going to an intermediate state $I$ before getting from $A$ to $B$:

If $A$→$I$ occurs with rate $k_1$ and $I$→$B$ with rate $k_2$ (no branching or backflow), then the **expected time** to go from $A$ to $B$ is the sum of expected times for the two steps:
$$
\tau = \frac{1}{k_1} + \frac{1}{k_2}.
$$
Thus the **effective rate** (one over the mean time) is
$$
k_{\mathrm{eff}} = \frac{1}{\tau} = \frac{1}{\frac{1}{k_1} + \frac{1}{k_2}} = \frac{k_1 k_2}{k_1 + k_2}.
$$

**Caveat:** This is an approximation under certain assumptions, but to first approximation accurate. This expression is clearly dominated by the **slower** rate, corresponding to the higher barrier. The only time that is not the case is when multiple barriers are close in energy, but even then the most important thing to remember still applies; we can think of an effective rate and barrier. This generalizes, so that we can always think of it as such, no matter how many barriers.

In [ ]:

#@title  Serial barriers: numerical exploration
import numpy as np
import matplotlib.pyplot as plt

def serial_keff(k1, k2):
    tau = 1.0/k1 + 1.0/k2
    return 1.0/tau

k2 = 10.0  # s^-1 (fixed)
ks = np.logspace(-2, 2, 200)  # sweep k1
keff_serial = [serial_keff(k1, k2) for k1 in ks]

plt.figure(figsize=(6,4))
plt.loglog(ks, keff_serial, label=f'k2={k2} s^-1')
plt.xlabel('k1 (s^-1)')
plt.ylabel('k_eff (s^-1)')
plt.title('Serial barriers: effective rate vs k1 (log-log)')
plt.grid(True, which='both', ls=':', alpha=0.4)
plt.legend()
plt.show()

# Print a couple of values to show limiting behavior
for k1 in [0.01, 0.1, 1.0, 10.0, 100.0]:
    print(f'k1={k1:6.2f}, k2={k2:6.2f} => k_eff={serial_keff(k1,k2):.4f} s^-1')


## Parallel barriers (competing pathways)

If A can go to B via two independent channels with rates \(k_1\) and \(k_2\), the time to leave A
and reach B is the minimum of two independent exponential waiting times. The minimum of independent exponentials
is exponential with rate equal to the sum of the rates, hence:
$$
k_{\mathrm{eff}} = k_1 + k_2.
$$

**Branching ratio:** the fraction of transitions taking path 1 is
$$
P(\text{path 1}) = \frac{k_1}{k_1 + k_2},
$$
and similarly for path 2. This means that we effectivley lower the barrier by having two options! Not too surprising perhaps, but again, the important part is that we can think of an effective rate even in the prescence of multiple parallel pathways to go from $A$to $B$, as in the folded-unfolded transtion.



In [ ]:

#@title Parallel barriers: numerical exploration
import numpy as np
import matplotlib.pyplot as plt

def parallel_keff(k1, k2):
    return k1 + k2

k2 = 10.0  # s^-1 fixed
ks = np.logspace(-2, 2, 200)
keff_parallel = [parallel_keff(k1, k2) for k1 in ks]

plt.figure(figsize=(6,4))
plt.semilogx(ks, keff_parallel, label=f'k2={k2} s^-1')
plt.xlabel('k1 (s^-1)')
plt.ylabel('k_eff (s^-1)')
plt.title('Parallel barriers: effective rate vs k1 (semi-log)')
plt.grid(True, ls=':', alpha=0.4)
plt.legend()
plt.show()

# branching fraction examples
for k1 in [0.01, 0.1, 1.0, 10.0, 100.0]:
    frac1 = k1 / (k1 + k2)
    print(f'k1={k1:6.2f}, k2={k2:6.2f} => k_eff={parallel_keff(k1,k2):.4f} s^-1, frac path1={frac1:.3f}')


## Beyond Temperature: Other Factors Affecting Rates

So far, we've focused on how temperature affects reaction rates through the Arrhenius equation and energy barriers. However, other factors can also significantly influence reaction kinetics by changing the stability of different states (reactant, product, and transition state).

One common factor used in protein folding studies is **chemical denaturant** (like urea or guanidine hydrochloride). Denaturants destabilize the folded state relative to the unfolded state, and can also affect the stability of the transition state.

By measuring reaction rates (like folding and unfolding rates) at different denaturant concentrations, we can gain insight into how the stability of the transition state is coupled to the folding/unfolding process. This leads us to the **Chevron plot**, which analyzes how folding and unfolding rates change with denaturant concentration.

In the context of our multi-state systems, folding and unfolding can often be viewed as **competing pathways** (parallel barriers). The observed rate is the sum of the rates of these competing processes. A Chevron plot helps visualize how the dominance shifts between folding and unfolding as the denaturant concentration changes, revealing information about the energy landscape and the position of the transition state along the reaction coordinate.

* * *

### Interactive Chevron and Energy Schematic

Below is an interactive plot showing a synthetic Chevron plot and a corresponding energy schematic. Use the sliders to explore how changing different parameters affects the folding and unfolding rates and the energy landscape.

### Questions

**Question 3**  
> If you increase the pre-exponential factor $k_0$, does the entire Chevron shift up or down?  
> What does that imply about the intrinsic timescale of folding/unfolding?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Increasing $k_0$ shifts the entire plot vertically upward (larger $k$ everywhere).  
  This reflects a faster intrinsic attempt frequency, i.e. the system tries barrier crossings more often.  
  The shape and position of the Chevron minimum are unaffected.
  </p>
</details>

---

**Question 4**  
> How does changing $\Delta G_N$ (folded vs unfolded stability at zero denaturant) move the position of the Chevron minimum along the denaturant axis?  
> What does this reveal about the balance point between folding and unfolding?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Increasing $\Delta G_N$ (making the folded state more stable at $D=0$) shifts the Chevron minimum to higher denaturant concentration.  
  The balance point moves because more denaturant is required to destabilize the folded state until folding and unfolding rates match.  
  This reveals how the minimum encodes relative native stability.
  </p>
</details>

---

**Question 5**  
> If you raise $\Delta G_{TS}$ (transition state relative to unfolded), which arm of the Chevron changes most — folding or unfolding?  
> How does this connect to the transition state’s role in determining unfolding rates?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Raising $\Delta G_{TS}$ increases the barrier for unfolding, so the unfolding arm shifts downward.  
  Folding is less affected, since the folded-to-TS barrier is not directly changed.  
  This shows that the TS energy mainly controls the rate of unfolding.
  </p>
</details>

---

**Question 6**  
> What happens to the folding arm’s slope when you increase $m_f$?  
> What about the unfolding arm when you increase $m_u$?  
> How do these slopes tell you how sensitive the transition state is to denaturant compared to the folded/unfolded states?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Increasing $m_f$ steepens the folding arm: folding becomes more sensitive to denaturant concentration.  
  Increasing $m_u$ steepens the unfolding arm: unfolding responds more strongly to denaturant.  
  Together, $m_f$ and $m_u$ indicate where the TS lies along the folding coordinate: if $m_f$ is large, the TS resembles the folded state more; if $m_u$ is large, it resembles the unfolded state more.
  </p>
</details>

---

**Question 7**  
> When you adjust $D_{schematic}$, how do the energies of the unfolded and transition states shift relative to the folded baseline?  
> How does the marker on the Chevron plot line up with the energy schematic?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Increasing $D_{schematic}$ lowers the unfolded and TS energies relative to the folded baseline (which is fixed at 0).  
  The marker on the Chevron plot moves along the curve at the same denaturant concentration, directly linking the kinetic data to the schematic energy landscape.  
  This shows how the energy levels encode the arms of the Chevron plot.
  </p>
</details>

### (Optional) challenges

**Challenge 1**  
> Adjust parameters so the Chevron minimum occurs at ~3 M denaturant.  
> Which parameter(s) did you need to change? Why?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Shifting the minimum mostly requires adjusting $\Delta G_N$, the native state stability at zero denaturant.  
  Making the folded state more stable pushes the balance point to higher denaturant concentrations.  
  You can also fine-tune $m_f$ and $m_u$ to adjust the slopes if needed.
  </p>
</details>

---

**Challenge 2**  
> Create conditions where folding is much slower than unfolding across the full denaturant range.  
> Which energy levels or slopes control this?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  To slow folding, raise the barrier between the unfolded and transition state (increase $\Delta G_{TS}$ or decrease $m_f$).  
  This reduces $k_f$ everywhere.  
  The unfolding barrier remains relatively small, so $k_u$ dominates.
  </p>
</details>

---

**Challenge 3**  
> Tune the parameters so the Chevron arms become nearly parallel (folding and unfolding respond similarly to denaturant).  
> What does this imply about the position of the transition state along the folding coordinate?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Making $m_f$ and $m_u$ similar in value gives parallel arms.  
  This implies that the transition state is roughly midway between the folded and unfolded states in terms of denaturant sensitivity.  
  In other words, the TS has features of both states equally.
  </p>
</details>

---

**Challenge 4**  
> Try to make a plot where the folded state is never favored (the Chevron minimum sits very high).  
> What happens to the schematic in this case?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  Reducing $\Delta G_N$ (weakening the native state) or increasing $\Delta G_{TS}$ destabilizes the folded state.  
  The Chevron minimum shifts upward, and folding rates remain low at all denaturant concentrations.  
  In the schematic, the folded basin lies above the unfolded one, so the system prefers the unfolded state regardless of $D$.
  </p>
</details>



### ϕ-Values and Chevron Plots

**Context:**  
Chevron plots give us folding ($k_f$) and unfolding ($k_u$) rate constants as a function of denaturant concentration.  
From the slopes of the folding and unfolding arms, we can learn how the transition state responds to denaturant relative to the folded and unfolded states.  
This idea generalizes to other perturbations too (mutations, temperature, pressure, etc.).

---

**What is a ϕ-value?**

ϕ-value analysis is used in *protein folding* to determine how much of a given residue’s interactions are already formed in the transition state.  

For a point mutation:
$$
\phi = \frac{\Delta \Delta G^{\ddagger}}{\Delta \Delta G^{\text{N-U}}}
$$

- $\Delta \Delta G^{\ddagger}$ = change in activation free energy (barrier height) caused by the mutation.  
- $\Delta \Delta G^{\text{N-U}}$ = change in equilibrium stability (folded vs unfolded free energy difference) caused by the mutation.  


**Clarifying question**  
> What is a ΔΔG?  

<details>
  <summary>Answer (click to expand)</summary>
  <p>
  A ΔΔG is the *difference of differences* in free energy.  
  It compares how a perturbation (like a mutation, denaturant, or temperature change) alters the free energy difference between two states.  

  Formally:  
  - First, define the free energy difference between two states (e.g. folded vs unfolded) as ΔG.  
  - Apply a perturbation and measure the new free energy difference, ΔG'.  
  - The change caused by the perturbation is  
    $$\Delta \Delta G = \Delta G' - \Delta G.$$

  Example: If a mutation destabilizes the folded state by 2 kJ/mol, then ΔΔG for folding = +2 kJ/mol.  
  In ϕ-value analysis, one ΔΔG refers to the change in the *barrier* (transition state vs unfolded), and another refers to the change in the *equilibrium stability* (native vs unfolded).
  </p>
</details>

---

**Interpretation:**

- $\phi \approx 1$: the mutation destabilizes the transition state as much as the native state → the residue is *structured* in the transition state.  
- $\phi \approx 0$: the mutation affects only the native state, not the transition state → the residue is *unstructured* in the transition state.  
- $0 < \phi < 1$: the residue is partially structured in the transition state.  

---

**Connection to Chevron Plots**

From chevron data:
- The **folding arm slope** ($m_f$) reflects how much the folding barrier changes with denaturant.  
- The **unfolding arm slope** ($m_u$) reflects how much the unfolding barrier changes.  

These slopes tell us how sensitive the transition state is relative to the folded and unfolded states.  
This is conceptually the same as ϕ-value analysis with mutations: we compare how a perturbation changes the **barrier** vs the **equilibrium stability**.  

Thus:
- **Chevron plot experiments** give global information about transition state position along the folding coordinate (via $m_f$ and $m_u$).  
- **ϕ-value analysis** (using mutations) gives residue-level detail about which parts of the protein are structured in the transition state.  

Together, they let us reconstruct the folding transition state ensemble.

In [ ]:
#@title Interactive Chevron and Energy Schematic
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

def piecewise_landscape(x_min=-1, x_max=1, barrier_x=0,
                        unfolded_y=0, folded_y=-5, barrier_y=5, # Updated default values for schematic
                        plateau_width=0.4):
    """
    Creates a piecewise linear folding landscape with flat minima and barrier.
    """
    # Define regions
    left_x = [x_min-1, x_min, x_min+plateau_width]
    left_y = [unfolded_y, unfolded_y, unfolded_y]

    barrier_xs = [barrier_x-plateau_width, barrier_x, barrier_x+plateau_width]
    barrier_ys = [barrier_y, barrier_y, barrier_y]

    right_x = [x_max-plateau_width, x_max, x_max+1]
    right_y = [folded_y, folded_y, folded_y]

    # Connect with linear ramps
    xs = np.array([
        left_x[0], left_x[1], left_x[2],
        barrier_xs[0], barrier_xs[1], barrier_xs[2],
        right_x[0], right_x[1], right_x[2]
    ])
    ys = np.array([
        left_y[0], left_y[1], left_y[2],
        barrier_ys[0], barrier_ys[1], barrier_ys[2],
        right_y[0], right_y[1], right_y[2]
    ])
    return xs, ys


def interactive_chevron_schematic_updated(Dmax=6, k0=1e3,
                                         deltaG_N=5, deltaG_TS=10, # Updated parameters
                                         m_f=0.7, m_u=0.8,
                                         D_schematic=0.0):
    """
    Generates interactive Chevron and energy schematic plots with updated parameters.

    Args:
        Dmax (float): Maximum denaturant concentration for the Chevron plot.
        k0 (float): Pre-exponential factor.
        deltaG_N (float): Free energy difference between folded and unfolded states at zero denaturant (Unfolded - Folded).
        deltaG_TS (float): Free energy of the transition state at zero denaturant (relative to Unfolded at 0).
        m_f (float): Dependence of folding barrier on denaturant.
        m_u (float): Dependence of unfolding barrier on denaturant.
        D_schematic (float): Denaturant concentration for the energy schematic.
    """
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

    # --- Chevron Plot (ax1) ---
    D = np.linspace(0, Dmax, 200)
    R, T = 0.001987, 298  # kcal/mol/K

    # Calculate folding and unfolding rates based on updated parameters
    # Assuming Unfolded state at 0 kcal/mol
    E_unfolded_0 = 0.0
    E_folded_0 = -deltaG_N # Folded state energy relative to unfolded at D=0
    E_ts_0 = deltaG_TS # Transition state energy relative to unfolded at D=0

    # Energy levels at concentration D
    E_unfolded_D = E_unfolded_0 - m_u * D  # Denaturant destabilizes unfolded (relative to a reference)
    E_folded_D = E_folded_0 # Assuming folded state stability less affected by denaturant directly (or included in m_f)
    E_ts_D = E_ts_0 + m_f * D # Denaturant stabilizes TS relative to folded (or destabilizes TS relative to unfolded)


    # Folding barrier (TS - Folded)
    DeltaG_f_D = E_ts_D - E_folded_D
    # Unfolding barrier (TS - Unfolded)
    DeltaG_u_D = E_ts_D - E_unfolded_D

    kf =  k0 * np.exp(-DeltaG_f_D/(R*T))
    ku =  k0 * np.exp(-DeltaG_u_D/(R*T))

    # Calculate observed rate
    kobs = kf + ku

    # Calculate D_min (where kf = ku, i.e., DeltaG_f_D = DeltaG_u_D)
    # E_ts_D - E_folded_D = E_ts_D - E_unfolded_D
    # -E_folded_D = -E_unfolded_D
    # -(E_folded_0) = -(E_unfolded_0 - m_u * D_min)
    # -(-deltaG_N) = -(0 - m_u * D_min)
    # deltaG_N = m_u * D_min
    # This assumes m_f term is factored into deltaG_TS or deltaG_N definition.
    # A more general derivation from kf=ku:
    # k0 * exp(-(deltaG_TS + m_f*D)/(R*T)) = k0 * exp(-(deltaG_TS - deltaG_N - m_u*D)/(R*T))
    # -(deltaG_TS + m_f*D) = -(deltaG_TS - deltaG_N - m_u*D)
    # deltaG_TS + m_f*D = deltaG_TS - deltaG_N - m_u*D
    # m_f*D = -deltaG_N - m_u*D
    # (m_f + m_u)*D = -deltaG_N  -> This seems incorrect based on typical Chevron.

    # Let's recalculate the barriers based on the energy levels relative to Unfolded at D=0
    # E_folded(D) = -deltaG_N_0 + m_f' * D  (assuming folded state stability also changes with D)
    # E_unfolded(D) = 0 + m_u' * D
    # E_TS(D) = deltaG_TS_0 + m_TS' * D

    # A more standard approach for Chevron is to define m-values relative to the transition state:
    # DeltaG_folding(D) = DeltaG_folding_0 - m_f * D
    # DeltaG_unfolding(D) = DeltaG_unfolding_0 - m_u * D
    # kf = k0 * exp(-DeltaG_folding(D) / RT)
    # ku = k0 * exp(-DeltaG_unfolding(D) / RT)

    # Let's use deltaG_N as DeltaG_F_U_0 (Free energy difference between Folded and Unfolded at D=0)
    # Let's use deltaG_TS as DeltaG_TS_U_0 (Free energy difference between TS and Unfolded at D=0)

    DeltaG_F_U_0 = -deltaG_N # deltaG_N is Unfolded - Folded, so Folded - Unfolded is -deltaG_N
    DeltaG_TS_U_0 = deltaG_TS

    DeltaG_TS_F_0 = DeltaG_TS_U_0 - DeltaG_F_U_0 # Energy of TS relative to Folded at D=0

    # Chevron equations with m-values relative to TS
    kf = k0 * np.exp(-(DeltaG_TS_F_0 - m_f * D) / (R*T))
    ku = k0 * np.exp(-(DeltaG_TS_U_0 + m_u * D) / (R*T))


    kobs = kf + ku

    # D_min: kf = ku => DeltaG_TS_F_0 - m_f * D_min = DeltaG_TS_U_0 - m_u * D_min
    # m_u * D_min - m_f * D_min = DeltaG_TS_U_0 - DeltaG_TS_F_0
    # (m_u - m_f) * D_min = (DeltaG_TS_U_0) - (DeltaG_TS_U_0 - DeltaG_F_U_0)
    # (m_u - m_f) * D_min = DeltaG_F_U_0
    # (m_u - m_f) * D_min = -deltaG_N
    # D_min = -deltaG_N / (m_u - m_f) -- This requires m_u > m_f for a positive D_min

    # Let's use the definition from the previous cell's formula for D_min, which is based on the intersection of the lines:
    # log(kf) = log(ku)
    # log(k0) - (deltaG_f0 + m_f*D)/(R*T) = log(k0) - (deltaG_u0 - m_u*D)/(R*T)
    # deltaG_f0 + m_f*D = deltaG_u0 - m_u*D
    # m_f*D + m_u*D = deltaG_u0 - deltaG_f0
    # (m_f + m_u)D = deltaG_u0 - deltaG_f0
    # D_min = (deltaG_u0 - deltaG_f0) / (m_f + m_u)

    # We need to relate deltaG_f0 and deltaG_u0 to deltaG_N and deltaG_TS
    # At D=0:
    # deltaG_f0 = Energy of TS relative to Folded = DeltaG_TS_F_0 = DeltaG_TS_U_0 - DeltaG_F_U_0 = deltaG_TS - (-deltaG_N) = deltaG_TS + deltaG_N
    # deltaG_u0 = Energy of TS relative to Unfolded = DeltaG_TS_U_0 = deltaG_TS

    # So, D_min = (deltaG_TS - (deltaG_TS + deltaG_N)) / (m_f + m_u) = -deltaG_N / (m_f + m_u)
    # This D_min formula seems consistent with the intersection point of the log rate lines.

    D_min = -deltaG_N / (m_f + m_u)


    # Plot rates on ax1
    ax1.plot(D, np.log10(kf), '--', label='Folding')
    ax1.plot(D, np.log10(ku), '--', label='Unfolding')
    ax1.plot(D, np.log10(kobs), '-', label='k_obs')

    # Add D_min line
    ax1.axvline(D_min, color='gray', linestyle=':', label=f'D_min ≈ {D_min:.2f} M')

    # Add marker for schematic D value
    ax1.axvline(D_schematic, color='red', linestyle='--', label=f'Schematic D = {D_schematic:.1f} M')


    # Set labels and title for ax1
    ax1.set_xlabel('Denaturant [M]')
    ax1.set_ylabel('log10(rate) [s$^{-1}$]')
    ax1.set_title('Synthetic Chevron Plot (Updated Parameters)')
    ax1.legend()
    ax1.grid(True)
    ax1.set_xlim(0, Dmax) # Set x-limits for Chevron plot

    DeltaG_TS_F_0 - m_f * D
    DeltaG_TS_U_0 + m_u * D
    # --- Energy Schematic (ax2) ---
    # Calculate energy levels at D_schematic based on updated parameters
    E_unfolded_level_schematic = 0.0
    E_folded_level_schematic = DeltaG_TS_U_0 - DeltaG_TS_F_0 + (m_u + m_f) * D_schematic # Corrected dependence based on Chevron math
    E_ts_level_schematic = DeltaG_TS_U_0 + m_u * D_schematic # Corrected dependence based on Chevron math

    # Generate piecewise landscape points
    # Using fixed x-coordinates for the schematic but adjusting y-levels
    x_schematic, y_schematic = piecewise_landscape(
        x_min=-1, x_max=1, barrier_x=0,
        unfolded_y=E_unfolded_level_schematic,
        folded_y=E_folded_level_schematic,
        barrier_y=E_ts_level_schematic,
        plateau_width=0.4
    )

    # Plot piecewise landscape
    ax2.plot(x_schematic, y_schematic, '-k', linewidth=2, label="Energy Profile")
    ax2.fill_between(x_schematic, y_schematic, min(y_schematic)-0.5, color="lightgray", alpha=0.5)


    # Add annotations for energy levels - Corrected positions
    ax2.text(1, E_folded_level_schematic, f' Folded ({E_folded_level_schematic:.1f})', va='bottom', ha='center') # Moved Folded label to the right
    ax2.text(-1, E_unfolded_level_schematic, f' Unfolded ({E_unfolded_level_schematic:.1f})', va='bottom', ha='center') # Moved Unfolded label to the left
    ax2.text(0, E_ts_level_schematic, f' TS ({E_ts_level_schematic:.1f})', va='bottom', ha='center')


    # Add annotations for energy barriers - these should match the DeltaG_f_D and DeltaG_u_D calculated for kinetics
    DeltaG_f_schematic = E_ts_level_schematic - E_folded_level_schematic
    DeltaG_u_schematic = E_ts_level_schematic - E_unfolded_level_schematic


    # Corrected arrow and text positions
    ax2.annotate('', xy=(1, E_folded_level_schematic), xytext=(0, E_ts_level_schematic),
                 arrowprops=dict(arrowstyle='<->', color='red', lw=1.5))
    ax2.text(0.5, (E_ts_level_schematic + E_folded_level_schematic) / 2, f' ΔG$_f$={DeltaG_f_schematic:.1f}',
             va='center', ha='center', color='red')

    ax2.annotate('', xy=(-1, E_unfolded_level_schematic), xytext=(0, E_ts_level_schematic),
                 arrowprops=dict(arrowstyle='<->', color='purple', lw=1.5))
    ax2.text(-0.5, (E_ts_level_schematic + E_unfolded_level_schematic) / 2, f' ΔG$_u$={DeltaG_u_schematic:.1f}',
             va='center', ha='center', color='purple')


    # Set labels and title for ax2
    ax2.set_xlabel('Reaction Coordinate (arbitrary)')
    ax2.set_ylabel('Energy [kcal/mol]')
    ax2.set_title(f'Energy Schematic at D = {D_schematic:.1f} M')
    ax2.grid(True)
    ax2.set_xlim(-2, 2) # Set x-limits
    # Adjust y-limits dynamically based on the levels
    min_y = min(E_folded_level_schematic, E_unfolded_level_schematic, E_ts_level_schematic)
    max_y = max(E_folded_level_schematic, E_unfolded_level_schematic, E_ts_level_schematic)
    ax2.set_ylim(min_y - 2, max_y + 2)


    fig.tight_layout()
    plt.show()

# Interactive sliders
interact(interactive_chevron_schematic_updated,
         Dmax=FloatSlider(min=4, max=8, step=0.1, value=6, description='Dmax'),
         k0=FloatSlider(min=100, max=5000, step=100, value=1000, description='k0'),
         deltaG_N=FloatSlider(min=-5, max=10, step=0.5, value=5, description='ΔG_N (U-F)'),
         deltaG_TS=FloatSlider(min=0, max=15, step=0.5, value=10, description='ΔG_TS (TS-U)'),
         m_f=FloatSlider(min=0.1, max=1.5, step=0.05, value=0.7, description='m_f'),
         m_u=FloatSlider(min=0.1, max=1.5, step=0.05, value=0.8, description='m_u'),
         D_schematic=FloatSlider(min=0, max=6, step=0.1, value=0.0, description='D_schematic'))